In [1]:
%run imports.ipynb
%run '08222019_functions.ipynb'
import allel
import zarr
import pandas as pd
import petl as etl
import numpy as np
import matplotlib.pyplot as plt
import dask.array as da
import seaborn as sns
from IPython.display import Image
palette = sns.color_palette()

imports_20150407.ipynb:26: MatplotlibDeprecationWarning: 
The savefig.jpeg_quality rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  "outputs_hidden": false


In [2]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=30)
cluster

In [3]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://10.35.213.36:46813 Dashboard: /user/carlo%20mariade%20marco1/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
calldata_out= zarr.open('../data/outgroup_alleles_phase2.zarr/')
calldata_biallel= zarr.open('../data/phase2_biallel_allele_count.zarr/')

----------------------------------------------------

## Merge 3L and 3R

In [3]:
metadata = pd.read_csv("samples.meta.txt", sep="\t")
metadata.head()

,ox_code,src_code,population,country,location,site,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude
0,AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95033368,30.99,ERS311878,5.60858,-1.54926
1,AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95843804,31.70,ERS311886,5.60858,-1.54926
2,AA0042-C,Takoradi_C7,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,107420666,35.65,ERS311894,4.91217,-1.77397
3,AA0043-C,Takoradi_H8,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,95993752,29.46,ERS311902,4.91217,-1.77397
4,AA0044-C,Takoradi_D10,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,103044262,33.67,ERS311910,4.91217,-1.77397


In [4]:
metadata = metadata[metadata.population != 'AOcol'] 
metadata = metadata[metadata.population != 'CMgam'] 
metadata = metadata[metadata.population != 'FRgam'] 
metadata = metadata[metadata.population != 'GAgam'] 
metadata = metadata[metadata.population != 'GQgam'] 
metadata = metadata[metadata.population != 'KE'] 
metadata = metadata[metadata.population != 'UGgam'] 
metadata.head()

,ox_code,src_code,population,country,location,site,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude
0,AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95033368,30.99,ERS311878,5.60858,-1.54926
1,AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95843804,31.70,ERS311886,5.60858,-1.54926
2,AA0042-C,Takoradi_C7,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,107420666,35.65,ERS311894,4.91217,-1.77397
3,AA0043-C,Takoradi_H8,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,95993752,29.46,ERS311902,4.91217,-1.77397
4,AA0044-C,Takoradi_D10,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,103044262,33.67,ERS311910,4.91217,-1.77397


In [5]:
pop_selection = metadata.population.isin({'GHcol', 'GHgam', 'BFgam', 'BFcol', 'GM', 'GW', 'GNgam', 'GNcol',
       'CIcol'}).values

In [6]:
pos_3L = allel.SortedIndex(callset_biallel["3L"]["variants/POS"])
region_3L = pos_3L.locate_range(18_000_000, 41_000_000)
#region_3L = pos_3L[region_3L]

In [7]:
g_3L = allel.GenotypeDaskArray(callset_biallel['3L']['calldata/GT'])
g_3L

<GenotypeDaskArray shape=(7897666, 1142, 2) dtype=int8>

In [ ]:
g_3L = allel.GenotypeDaskArray(callset_biallel['3L']['calldata/GT'][region_3L]).subset(sel1=pop_selection)
g_3L

In [ ]:
pos_3R = allel.SortedIndex(callset_biallel["3R"]["variants/POS"])
region_3R = pos_3R.locate_range(2_000_000, 3_000_000)
#region_3R = pos_3R[region_3R]

In [ ]:
g_3R = allel.GenotypeChunkedArray(callset_biallel['3R']['calldata/GT'][region_3R]).subset(sel1=pop_selection)
g_3R

In [ ]:
root = zarr.open('outgroup_alleles_phase2.zarr', mode='a')